## V4は自動でデータの組み合わせを変更し、自動で入力系列数と出力系列数を変更するプログラムです。ただし、未完成

## ライブラリのインポート

In [1]:
from keras.models import Sequential
from keras.utils import np_utils, plot_model
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.initializers import glorot_uniform, orthogonal, TruncatedNormal
from keras.callbacks import EarlyStopping
from keras.layers.recurrent import GRU, SimpleRNN

import pandas as pd #行列計算
import numpy as np #行列計算
import math #数値計算
import itertools #順列・組み合わせ
import time
import talib as ta # テクニカル指標

import matplotlib.pyplot as plt #グラフ
import winsound # ビープ音
import os

# マルチプロセス
from multiprocessing import Process
from multiprocessing import Queue

from df_method import rise_fall_rate, moving_average, GCDC, df_shift, add_data, RSI, Z_score_normalization, Min_Max_normalization

Using TensorFlow backend.


## モデル構築

In [2]:
class Prediction :
    #初期化
    def __init__(self, maxlen, n_hidden, n_in, n_out, learning_model):
        self.maxlen = maxlen #入力系列数
        self.n_hidden = n_hidden #出力次元（隠れ層内のニューロン数）
        self.n_in = n_in #学習データの列数
        self.n_out = n_out #ラベルデータの列数
        
        self.learning_model = learning_model #●学習モデルの選択

    #モデルの生成
    def create_model(self):
        model = Sequential()
        if self.learning_model == 'RNN':
            #RNN層
            model.add(SimpleRNN(self.n_hidden,
                                batch_input_shape = (None, self.maxlen, self.n_in),
                                kernel_initializer = glorot_uniform(seed=20170719),
                                recurrent_initializer = orthogonal(gain=1.0, seed=20170719),
                                dropout = 0.5,
                                recurrent_dropout = 0.5))
        elif self.learning_model == 'LSTM':
            #LSTM層
            model.add(LSTM(self.n_hidden,
                           batch_input_shape = (None, self.maxlen, self.n_in),
                           kernel_initializer = glorot_uniform(seed=20170719), 
                           recurrent_initializer = orthogonal(gain=1.0, seed=20170719), 
                           dropout = 0.5, 
                           recurrent_dropout = 0.5))
        elif self.learning_model == 'GRU':
            #GRU層
            model.add(GRU(self.n_hidden,
                          batch_input_shape = (None, self.maxlen, self.n_in),
                          kernel_initializer = glorot_uniform(seed=20170719),
                          recurrent_initializer = orthogonal(gain=1.0, seed=20170719),
                          dropout = 0.5,
                          recurrent_dropout = 0.5))
        #ドロップアウト層
        model.add(Dropout(0.5))
        #結合層
        model.add(Dense(self.n_out, kernel_initializer = glorot_uniform(seed=20170719)))
        #活性化層
        model.add(Activation("softmax"))
        #コンパイル
        model.compile(loss="categorical_crossentropy", optimizer = "Adam", metrics = ['categorical_accuracy']) # "RMSprop"
        return model

    # 学習
    def train(self, x_train, t_train, batch_size, epochs) :
        early_stopping = EarlyStopping(patience=0, verbose=1)
        model = self.create_model()
        self.hist = model.fit(x_train, t_train, batch_size = batch_size, epochs = epochs, verbose = 1,
                              shuffle = True, callbacks = [early_stopping], validation_split = 0.1)
        return model

## 変数宣言

In [3]:
is_debug = True
csv_path = './csv/'
x_days_later = 1 # '1', '2', '7', '30', '365'
learning_model = 'GRU' #['RNN', 'LSTM', 'GRU']
start_date = '2009-3-10' #  '1960', '1970', '1980', '1990', '2000', '2010'
end_date = '2018-10-31'# 終点年月日

min_maxlen = 100
max_maxlen = 1000
min_n_hidden = 100
max_n_hidden = 500

target_name = 'USD_JPY_diff'# 'nikkei_Close', 'nikkei_diff', 'USD_JPY'
max_score = 0 # 最高正答率

## 学習データの用意

In [9]:
df_list = {} # データフレームの辞書
df_list_name = ''

#ドル円の始値と終値の差
df = pd.read_csv(csv_path+'USD_JPY.csv', index_col='Date', parse_dates=True)#読み込み
df = df.apply(np.log)*100#正規化
df = df['USD_JPY_Close'] - df['USD_JPY_Open']#終値と始値の差を求める
df = df['1989-10-16':]#始値と終値と高値と安値が記録され始めた日からのみ抽出
df = df.rename('USD_JPY_diff')#名前を付ける
df = pd.DataFrame(df)# sriesからdataframeに変換
df_list['USD_JPY_diff'] = df

# 為替レート
df_list['USD_JPY'] = add_data(csv_path+'USD_JPY.csv', x_days_later)
#df_list['EUR_JPY'] = add_data(csv_path+'EUR_JPY.csv', x_days_later)
#df_list['EUR_USD'] = add_data(csv_path+'EUR_USD.csv', x_days_later)

# 株価指数
#df_list['nikkei'] = add_data(csv_path+'nikkei.csv', x_days_later)
#df_list['TOPIX'] = add_data(csv_path+'TOPIX.csv', x_days_later)
#df_list['DOW30'] = add_data(csv_path+'DOW30.csv', x_days_later)
#df_list['NASDAQ'] = add_data(csv_path+'NASDAQ.csv', x_days_later)

# 米国債
df = pd.read_csv(csv_path+'treasury_10.csv', index_col='Date', parse_dates=True)
df_list['treasury_10'] = rise_fall_rate(df, x_days_later)

# テクニカル指標を求めるために使用-----------------------------------------
df = pd.read_csv(csv_path+'USD_JPY.csv', index_col=0, parse_dates=True)
df_close = df['USD_JPY_Close']

## 全データフレームを結合

In [10]:
#全データフレームを結合
df_x = df_list[target_name] # 予測したいデータ
df_list_name = target_name # 予測したいデータの名前
for i, j in df_list.items(): #range(len(df_list) - 1):   
    if i == target_name:
        continue
    df_list_name = df_list_name + ' ' + i 
    df_x = df_x.join(j, how='inner', rsuffix='_' + i)

#指定の期間を抽出
if end_date == '':
    df_x = df_x[start_date:]
else:
    df_x = df_x[start_date:end_date]
    
print(df_x)

            USD_JPY_diff  USD_JPY_Open  USD_JPY_High  USD_JPY_Low  \
Date                                                                
2009-03-10     -0.182279      0.578359     -0.040339     0.010213   
2009-03-11     -1.418753     -0.172143     -0.303061    -0.882147   
2009-03-12      0.451283     -1.418753     -0.324281    -1.442457   
2009-03-13      0.326931      0.451283      0.131866     1.524847   
2009-03-16      0.091710      0.377918      0.000000     0.861014   
2009-03-17      0.416731      0.091710      0.323854     0.112205   
2009-03-18     -2.433260      0.416731     -0.131439    -2.477162   
2009-03-19     -1.792595     -2.402086     -2.282013    -2.261775   
2009-03-20      1.481042     -1.834914     -0.352551     0.649941   
2009-03-23      1.297843      1.251397      1.105326     1.580478   
2009-03-24      0.944469      1.308158      1.235403     1.318883   
2009-03-25     -0.378653      0.975117     -0.203149     0.010316   
2009-03-26      1.212754     -0.37

## ラベルデータの用意

In [11]:
# ラベルデータを作成する列
target = df_x[target_name]
#●デバッグ
            
# 空のデータフレーム作成
df_t = pd.DataFrame(index=df_x.index, columns=['jump', 'rise', 'fall', 'drop'])
df_t = df_t.fillna(0) #０で埋める

# 条件にあった値を置換する
df_t.loc[0.995033085 <= target, 'jump'] = 1
df_t.loc[(0 <= target) & (target < 0.995033085), 'rise'] = 1
df_t.loc[(-0.995033085 <= target) & (target < 0), 'fall'] = 1
df_t.loc[target < -0.995033085, 'drop'] = 1

# データを整える
df_t = df_t.shift(-1 * x_days_later, axis=0)#予測先日数分だけ縦にずらす
df_t = df_t.drop(df_t.index[-1*x_days_later:], axis=0)#ラベルデータ末尾の行を削除
df_x = df_x.drop(df_x.index[-1*x_days_later:], axis=0)#学習データの末尾の行を削除

print(df_t)

            jump  rise  fall  drop
Date                              
2009-03-10   0.0   0.0   0.0   1.0
2009-03-11   0.0   1.0   0.0   0.0
2009-03-12   0.0   1.0   0.0   0.0
2009-03-13   0.0   1.0   0.0   0.0
2009-03-16   0.0   1.0   0.0   0.0
2009-03-17   0.0   0.0   0.0   1.0
2009-03-18   0.0   0.0   0.0   1.0
2009-03-19   1.0   0.0   0.0   0.0
2009-03-20   1.0   0.0   0.0   0.0
2009-03-23   0.0   1.0   0.0   0.0
2009-03-24   0.0   0.0   1.0   0.0
2009-03-25   1.0   0.0   0.0   0.0
2009-03-26   0.0   0.0   1.0   0.0
2009-03-27   0.0   0.0   1.0   0.0
2009-03-30   1.0   0.0   0.0   0.0
2009-03-31   0.0   0.0   1.0   0.0
2009-04-01   0.0   1.0   0.0   0.0
2009-04-02   0.0   1.0   0.0   0.0
2009-04-03   0.0   1.0   0.0   0.0
2009-04-06   0.0   0.0   1.0   0.0
2009-04-07   0.0   0.0   1.0   0.0
2009-04-08   0.0   1.0   0.0   0.0
2009-04-09   0.0   0.0   1.0   0.0
2009-04-10   0.0   0.0   1.0   0.0
2009-04-13   0.0   0.0   0.0   1.0
2009-04-14   0.0   1.0   0.0   0.0
2009-04-15   0.0   0

## 独立したプロセス

In [15]:
def learning(q, maxlen, n_hidden, n_in, n_out, learning_model, x_train, t_train, batch_size, epochs):
    # モデル定義
    prediction = Prediction(maxlen, n_hidden, n_in, n_out, learning_model)
    # 学習
    model = prediction.train(x_train, t_train, batch_size, epochs)
    # データを送信
    q.put(model)

## メイン処理

In [16]:
# 結果を記録する変数
result_csv = pd.DataFrame(columns=['day', 'model', 'start date', 'end date', 'maxlen', 'n_hidden', 'combination', 'time', 'correct', 'semi_correct'])
debug_csv = pd.DataFrame(columns=['time', 'x_train.shape', 't_train.shape', 'batch size', 'epochs'])

# インデックスと列名を外す
x_data = df_x.values
t_data = df_t.values

maxlen = min_maxlen # 最小入力系列数
while maxlen <= max_maxlen:
    n_in = x_data.shape[1]   # 学習データ（＝入力）の列数
    n_out = t_data.shape[1]  # ラベルデータ（=出力）の列数
    len_seq = x_data.shape[0] - maxlen + 1
    print('len_seq', len_seq, '\n')#●デバッグ
    data = []
    target = []

    #
    for i in range(0, len_seq):
        data.append(x_data[i:i+maxlen, :])
        target.append(t_data[i+maxlen-1, :])

    #
    x = np.array(data).reshape(len(data), maxlen, n_in)
    t = np.array(target).reshape(len(data), n_out)

    #
    if is_debug == True:
        print('x.shape=', x.shape)
        print('t.shape=', t.shape, '\n')

    # ここからソースコードの後半
    n_train = int(len(data)*0.9)              # 訓練データ長
    x_train,x_test = np.vsplit(x, [n_train])  # 学習データを訓練用とテスト用に分割
    t_train,t_test = np.vsplit(t, [n_train])  # ラベルデータを訓練用とテスト用に分割

    #
    if is_debug == True:
        print('x_train.shape=', x_train.shape)
        print('x_test.shape=', x_test.shape, '\n')
        print('t_train.shape=', t_train.shape)
        print('t_test.shape=', t_test.shape, '\n')
            
    #メイン処理--------------------------------------------------------------
    n_hidden = min_n_hidden # 最小隠れ層ユニット数
    while n_hidden <= max_n_hidden:   
        epochs = 100      # エポック数（同じデータでの学習回数）
        batch_size = 256  #バッチサイズ
                
        #パラメータの表示
        print('maxlen: ', maxlen)
        print('n_hidden: ', n_hidden, '\n')
        
        # 学習時間の計測開始
        start = time.time()
        
        # モデル定義
        #prediction = Prediction(maxlen, n_hidden, n_in, n_out, learning_model)
                    
        # 学習
        #model = prediction.train(x_train, t_train, batch_size, epochs)
            
        # 学習の実行
        q = Queue()
        p = Process(target=learning, args=(q, maxlen, n_hidden, n_in, n_out, learning_model, x_train, t_train, batch_size, epochs,))
        p.start()
        model = q.get()
        p.join()
            
        # 学習時間の計測終了と表示
        end = time.time()

        #予測精度の評価------------------------------------------------------

        # 正答率、準正答率（騰落）集計
        preds = model.predict(x_test)
                
        #正解数を数える変数
        correct = 0
        semi_correct = 0
                
        #表を作るためのデータフレーム
        matrix = pd.DataFrame(columns=['jump!', 'rise!', 'fall!', 'drop!'], index=['jump?', 'rise?', 'fall?', 'drop?'])
        matrix = matrix.fillna(0)

        #正解数を数える
        for i in range(len(preds)):
            pred = np.argmax(preds[i,:])#argmaxとは配列の最大要素のインデックスを返すメソッドである
            tar = np.argmax(t_test[i,:])
            matrix.iat[pred, tar] = matrix.iat[pred, tar] + 1 #●マトリックスのセルをインクリメント
            if pred == tar :#完全一致
               correct += 1
            else :
                if pred+tar == 1 or pred+tar == 5 :
                    semi_correct += 1
                
        #正答率と準正答率を求める
        correct_rate = 1.0 *correct / len(preds) 
        semi_correct_rate = 1.0 * (correct+semi_correct) / len(preds)
                
        #csvに記録
        series = pd.Series([x_days_later,
                            learning_model,
                            start_date,
                            end_date,
                            maxlen,
                            n_hidden,
                            df_list_name,
                            int(end - start),
                            correct_rate,
                            semi_correct_rate],
                            index = result_csv.columns)
        result_csv = result_csv.append(series, ignore_index = True)
        result_csv.to_csv('./log_' + learning_model + '/' + df_list_name + '.csv', index=False)
        
        #csvに記録
        series2 = pd.Series([int(end - start),
                             x_train.shape,
                             t_train.shape,
                             batch_size,
                             epochs],
                             index = debug_csv.columns)
        debug_csv = debug_csv.append(series2, ignore_index = True)
        debug_csv.to_csv('./log_' + learning_model + '/debug.csv', index=False)
                
        #表と正答率と学習時間を表示
        print('\n', matrix, '\n')
        print("正答率:", 1.0 * correct / len(preds))
        print("準正答率（騰落）:", 1.0 * (correct+semi_correct) / len(preds))                
                
        #次のステップへ
        n_hidden += 100
                    
    #次のステップへ
    maxlen += 100

len_seq 2409 

x.shape= (2409, 100, 5)
t.shape= (2409, 4) 

x_train.shape= (2168, 100, 5)
x_test.shape= (241, 100, 5) 

t_train.shape= (2168, 4)
t_test.shape= (241, 4) 

maxlen:  100
n_hidden:  100 



BrokenPipeError: [Errno 32] Broken pipe

## 終了の合図

In [ ]:
for i in range(3):
    winsound.Beep(784,300)
    winsound.Beep(698,300)
    winsound.Beep(784,600)

## 最高正答率

In [ ]:
print('平均計算時間')
print(result_csv['time'].mean(), '秒')
    
print('max correct')
print(result_csv[result_csv['correct'] == result_csv['correct'].max()])
    
print('\nmax semi correct')
print(result_csv[result_csv['semi_correct'] == result_csv['semi_correct'].max()])

## 平均正答率

In [ ]:
print('正答率')
print(result_csv['correct'].mean())

print('準正答率')
print(result_csv['semi_correct'].mean())

## 結果を記録

In [ ]:
report_csv = pd.DataFrame(columns=['mean time', 'maxlen', 'n_hidden', 'max correct', 'max semi correct', 'mean correct', 'mean semi correct', 'combination'])

#result_csv = pd.read_csv('log/USD_JPY_diff nikkei.csv', index_col=0, parse_dates=True)

series3 = pd.Series([result_csv['time'].sum() / 60,
                     result_csv['maxlen'][result_csv['semi_correct'].idxmax()],
                     result_csv['n_hidden'][result_csv['semi_correct'].idxmax()],
                     result_csv['correct'].max(),
                     result_csv['semi_correct'].max(),
                     result_csv['correct'].mean(),
                     result_csv['semi_correct'].mean(),
                     df_list_name],
                        index = report_csv.columns)

report_csv = report_csv.append(series3, ignore_index = True)

print(report_csv)

if os.path.exists('log_' + learning_model + '/log.csv'):
    report_csv.to_csv('log_' + learning_model + '/log.csv', index=False, mode='a', header=False)
else:
    report_csv.to_csv('log_' + learning_model + '/log.csv', index=False, mode='a', header=True)
